In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train=pd.read_csv("train.csv", encoding='utf8')

In [ ]:
train.head()

,ID,keywords,age,sex
0,1,fibre:16;quoi:1;dangers:1;combien:1;hightech:1...,62.0,F
1,2,restaurant:1;marrakech.shtml:1,35.0,M
2,3,payer:1;faq:1;taxe:1;habitation:1;macron:1;qui...,45.0,F
3,4,rigaud:3;laurent:3;photo:11;profile:8;photopro...,46.0,F
4,5,societe:1;disparition:1;proche:1;m%c3%a9lanie....,42.0,F


In [ ]:
train=train[1:10000]

In [ ]:
len(train)

3058583

In [ ]:
train.isnull().sum()

ID               0
keywords    340459
age              1
sex              1
dtype: int64

In [ ]:
train=train.dropna()

In [ ]:
len(train)

2718123

In [ ]:
train.drop(['ID'],axis = 1,inplace = True)

In [ ]:
train.head()

,keywords,age,sex
0,fibre:16;quoi:1;dangers:1;combien:1;hightech:1...,62.0,F
1,restaurant:1;marrakech.shtml:1,35.0,M
2,payer:1;faq:1;taxe:1;habitation:1;macron:1;qui...,45.0,F
3,rigaud:3;laurent:3;photo:11;profile:8;photopro...,46.0,F
4,societe:1;disparition:1;proche:1;m%c3%a9lanie....,42.0,F


In [ ]:
keywords=train['keywords']

In [ ]:
def cleantext(keywords):
    final_text = ''
    for word_count in keywords.split(';'):
        word, count = word_count.split(':')
        count = int(count)
        if count <= 1:
            final_text += word + " "
        else:
            for i in range(1, count+1):
                final_text += word + " "
    return final_text

In [ ]:
def cleantext2(kw):
    try: 
        return cleantext(kw)
    except ValueError:
        return None

In [ ]:
train['keywords'] = train['keywords'].apply(lambda dc: cleantext2(dc))

In [ ]:
train=train.dropna()

In [ ]:
train.head()

,keywords,age,sex
0,fibre fibre fibre fibre fibre fibre fibre fibr...,62.0,F
1,restaurant marrakech.shtml,35.0,M
2,payer faq taxe habitation macron qui detail pr...,45.0,F
3,rigaud rigaud rigaud laurent laurent laurent p...,46.0,F
4,societe disparition proche m%c3%a9lanie.gonide...,42.0,F


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train,X_test, y_train, y_test=train_test_split(train["keywords"],train["sex"],test_size=0.2, random_state=42)

In [ ]:
X_train.head()

16644    homme maelys disparition place garde deuxieme ...
61579    people people people people people people peop...
27895    travaillait qui qui ingenieur les gagnants pou...
42062    fait garde qui vue deuxieme clients teste ses ...
68040                         jobs profile profile search 
Name: keywords, dtype: object

In [ ]:
reg=RandomForestClassifier (max_depth=50, random_state=0, min_samples_leaf=3,min_samples_split=5)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect=CountVectorizer()

In [ ]:
vect.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [ ]:
from sklearn import svm
svm = svm.SVC(C=10)
svm.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
X_test = vectorizer.transform(X_test)
y_pred = svm.predict(X_test)
print(confusion_matrix(y_test, y_pred))

[[492 409]
 [414 601]]


In [ ]:
def pred(msg):
    msg = vectorizer.transform([msg])
    prediction = svm.predict(msg)
    return prediction[0]

In [ ]:
pred("Je pratique mon francais avec ca google colabs notebook")

'M'

In [ ]:
test=pd.read_csv("test.csv", encoding='utf8')

In [ ]:
out=reg2.predict(test)

In [ ]:
np.unique(out, return_counts=True)
Results=pd.DataFrame({'Prob_Male': out[:, 2], 'Estim_Age': out[:, 3]})

In [ ]:
Final=pd.concat([train['ID'], train['keywords'], Results['Prob_Male'],Results['Estim_Age'], axis=1, keys=['ID', 'Keywords','Prediction Male','Estimated Age'])

In [ ]:
Final=Final.dropna()

In [ ]:
Final['ID']=Final['ID'].round().astype(int)

In [ ]:
Final.head()

In [ ]:
Final.to_csv('/Users/macbook/Desktop/ML with Python/Group Project/Group_Project-Tanasit.csv', index = False)